In [4]:
import sqlite3

In [15]:
appstore_table = 'appstore' 
query = "DROP TABLE IF EXISTS {}".format(appstore_table)
cursor = db.cursor()
cursor.execute(query)
db.commit()

In [16]:
DBName = 'opinion_outside.db'
db = sqlite3.connect(DBName, check_same_thread=False)
db.row_factory = sqlite3.Row
appstore_table = 'appstore'
appstore_columns = {
    'USER': 'text',
    'DATE': 'text PRIMARY KEY',
    'STAR': 'integer',
    'LIKE': 'integer',
    'TITLE': 'text',
    'REVIEW': 'text PRIMARY KEY'
}

In [17]:
cursor = db.cursor()
colum_info = ",".join(col_name + ' ' + col_type for col_name, col_type in appstore_columns.items())
query = "CREATE TABLE IF NOT EXISTS {} ({})".format(appstore_table, colum_info)
cursor.execute(query)
db.commit()

OperationalError: table "appstore" has more than one primary key

In [18]:
import pandas as pd
import xmltodict
import requests
import os


def get_url_index(url):
    response = requests.get(url).content.decode('utf8')
    xml = xmltodict.parse(response)

    last_url = [l['@href'] for l in xml['feed']['link'] if (l['@rel'] == 'last')][0]
    last_index = [int(s.replace('page=', '')) for s in last_url.split('/') if ('page=' in s)][0]

    return last_index

# https://stackoverflow.com/questions/1090282/api-to-monitor-iphone-app-store-reviews
def appstore_crawler(appid, outfile='./appstore_reviews.csv'):
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=%i/sortby=mostrecent/xml' % appid

    try:
        last_index = get_url_index(url)
    except Exception as e:
        print (url)
        print ('\tNo Reviews: appid %i' %appid)
        print ('\tException:', e)
        return

    result = list()
    for idx in range(1, 14+1):
        url = "https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=%i/sortby=mostrecent/xml?urlDesc=/customerreviews/id=%i/sortBy=mostRecent/xml" % (idx, appid, appid)
        print(url)

        response = requests.get(url).content.decode('utf8')
        try:
            xml = xmltodict.parse(response)
        except Exception as e:
            print ('\tXml Parse Error %s\n\tSkip %s :' %(e, url))
            continue

        try:
            num_reivews= len(xml['feed']['entry'])
        except Exception as e:
            print ('\tNo Entry', e)
            continue

        try:
            xml['feed']['entry'][0]['author']['name']
            single_reviews = False
        except:
            #print ('\tOnly 1 review!!!')
            single_reviews = True
            pass

        if single_reviews:
                result.append({
                    'USER': xml['feed']['entry']['author']['name'],
                    'DATE': xml['feed']['entry']['updated'],
                    'STAR': int(xml['feed']['entry']['im:rating']),
                    'LIKE': int(xml['feed']['entry']['im:voteSum']),
                    'TITLE': xml['feed']['entry']['title'],
                    'REVIEW': xml['feed']['entry']['content'][0]['#text'],
                })
        else:
            for i in range(len(xml['feed']['entry'])):
                result.append({
                    'USER': xml['feed']['entry'][i]['author']['name'],
                    'DATE': xml['feed']['entry'][i]['updated'],
                    'STAR': int(xml['feed']['entry'][i]['im:rating']),
                    'LIKE': int(xml['feed']['entry'][i]['im:voteSum']),
                    'TITLE': xml['feed']['entry'][i]['title'],
                    'REVIEW': xml['feed']['entry'][i]['content'][0]['#text'],
                })

    res_df = pd.DataFrame(result)
    res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S")
    #res_df.to_csv(outfile, encoding='utf-8-sig', index=False)
    res_df.to_sql('appstore', db, if_exists = 'append', index = False)
    print ('Save reviews to DB: appstore \n')
    return res_df


if __name__ == '__main__':
    # https://apps.apple.com/us/app/youtube-watch-listen-stream/id544007664
    app_id = 1248716281
    outfile = os.path.join('appstore_' + str(app_id)+'.csv')
    df = appstore_crawler(app_id, outfile=outfile)

https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=2/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=3/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=4/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=5/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=6/id=1248716281/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1248716281/sortBy=mostRecent/xml
https://itunes.apple.com/kr/rss/customerreviews/page=7/id=1248716281/sortby=mostrecent/x

In [19]:
query = "SELECT * FROM appstore"
result = pd.read_sql(query,db)

In [20]:
result

,USER,DATE,STAR,LIKE,TITLE,REVIEW
0,헤라씨네,2022-08-09 06:32:37-07:00,1,0,모바일 앱 바뀌고 너무 불편해요,"다른 증권사 대비 사용하기 엄청 편하고,\n좋았는데…\n멀쩡하게 잘 쓰고 있는데 왜..."
1,jjjjaateee,2022-08-09 06:29:03-07:00,3,0,오류가 너무많아요,관심종목상에서 등락률도 안나옵니다\n오늘나온 업데이트도 다해봤는데\n아직도 오류가 ...
2,자두맛엉덩이,2022-08-09 03:33:57-07:00,1,0,문의를 여러번 남겼는데 답이 없네요,갤탭 사용시 오른쪽에 피드 어떻게 없에나요? \n\n창이 작아져서 너무 불편해요
3,팡팡팡,2022-08-07 21:37:35-07:00,1,0,보기쉬웠던걸 꼬아놨네.,"무슨 프로그래머가 사용하는 것처럼 엄청 꼬아놨네. 사용자 늘었으니, 사용하지 말라는..."
4,daveane,2022-08-07 20:38:47-07:00,1,0,공인인증서 변경이 안되서 로그인이 안되네요,자동 업데이트 이후에 다른 공인인증서로 로그인하려고 선택하면 그 상태에서 더 이상 ...
...,...,...,...,...,...,...
495,모두다 처벌받기를,2021-03-29 09:06:57-07:00,2,0,어플 한번 터지고 업데이트하면 잘 되겠지.. 했는데,이건 뭐.. \n고치나 마나 입니다\n터지기 전에는 진짜 불만 아예없었거든요? \n...
496,영어공부홧팅,2021-03-28 22:55:23-07:00,2,0,업데이트 후 불편해진 어플,너무 불편해요 예전이 더 편했는데
497,매니악08,2021-03-28 00:58:55-07:00,1,0,이체오류 진짜 심함,며칠전 오류떠서 어플 삭제했다가 다시 하라고해서 했는데 지금 또 확인버튼 오류네요\...
498,Wrrrrrrrrrrrry,2021-03-27 22:25:36-07:00,1,0,이체 안됨,"은행으로 이체하려 하는데 보안카드, OTP 준비하라는 메시지 뜨고 반응 없음. \n..."
